In [1]:
#import libraries
import pandas as pd
import csv
import re

### Extracting two pandas DataFrames from the data.

In [2]:
with open("Excelfiles/Sample Input - Sample.csv.csv") as file:
    reader = csv.reader(file)
    data_1 = []
    data_2 = []
    
    for index,row in enumerate(reader,start = 1):
        if index <=20:
            data_1.append(row)
        else:
            data_2.append(row)
    
    
data1 = pd.DataFrame(data_1[1:],columns=data_1[0])
data2 = pd.DataFrame(data_2[1:],columns=data_2[0])

In [3]:
data1.head()

,ItemId,ComputerId,ItemName,Comment,Unit,C/S/ID,DataType,BitPos,Address,MAX,...,ScaleMin,Decimal,,,,,,,,
0,35827,10,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),,0,REAL : Float,0,0x16464,10000,...,-10000,0,,,,,,,,
1,35828,10,L_R1SDD_SPDREF,R1 Screw Down DS Speed Reference (mps),,0,REAL : Float,0,0x16468,10000,...,-10000,0,,,,,,,,
2,35829,10,L_R1SDL_SPDREF,R1 Screw Down LS Speed Reference (mps),,0,REAL : Float,0,0x1646c,10000,...,-10000,0,,,,,,,,
3,35830,10,L_R1SDW_SPDFBK,R1 Screw Down WS Speed Feedback (mps),,0,REAL : Float,0,0x16470,10000,...,-10000,0,,,,,,,,
4,35831,10,L_R1SDD_SPDFBK,R1 Screw Down DS Speed Feedback (mps),,0,REAL : Float,0,0x16474,10000,...,-10000,0,,,,,,,,


### Concatenate Date and Time series to create a new series and converting it to DateTime dtype.

In [4]:
data2.insert(0,'DateTime',pd.to_datetime(data2['Date'] + " " + data2['Time']),allow_duplicates=True)

### Dropping undesired series

In [5]:
data2.drop(['Date', 'Time', 'Milli Sec'],axis = 1,inplace=True)

### The input data is presented in a pivot table format. The data is transformed into flat table format.

In [6]:
data2 = data2.melt(id_vars=['DateTime']).sort_values('DateTime')

### Grouping data in 10sec interval.

In [7]:
data2 = data2.groupby([pd.Grouper(key='DateTime',freq='10s',origin='start'),'variable']).max().reset_index()

### Joining both dataframes on common key value.

In [8]:
data2 = data2.join(data1.set_index('ItemName'),how='left',on=['variable'])

### Extracting Units using regex and lambda function

In [9]:

data2['tag__unit'] = data2['Comment'].apply(lambda x: re.findall(r'\((.*?)\)',x)[-1] if len(re.findall(r'\((.*?)\)',x))>0 else '') 

In [10]:
data2.head()

,DateTime,variable,value,ItemId,ComputerId,Comment,Unit,C/S/ID,DataType,BitPos,...,Decimal,,,,,,,,,tag__unit
0,2022-06-05 21:58:33,L_R1SDD_SPDFBK,0,35831,10,R1 Screw Down DS Speed Feedback (mps),,0,REAL : Float,0,...,0,,,,,,,,,mps
1,2022-06-05 21:58:33,L_R1SDD_SPDREF,0,35828,10,R1 Screw Down DS Speed Reference (mps),,0,REAL : Float,0,...,0,,,,,,,,,mps
2,2022-06-05 21:58:33,L_R1SDLS_SPDFBK,0,83537,10,R1 SCREW DOWN LS SPEED FBK (MPS),,0,REAL : Float,0,...,0,,,,,,,,,MPS
3,2022-06-05 21:58:33,L_R1SDLS_SPDREF,0,83534,10,R1 SCREW DOWN LOW SPEED REF (MPS),,0,REAL : Float,0,...,0,,,,,,,,,MPS
4,2022-06-05 21:58:33,L_R1SDL_SPDFBK,0,35832,10,R1 Screw Down LS Speed Feedback (mps),,0,REAL : Float,0,...,0,,,,,,,,,mps


In [11]:
data3 = pd.DataFrame()
data3['event_timestamp'] = data2['DateTime']
data3['tag__id'] = data2['ItemId']
data3['tag__name'] = data2['variable']
data3['tag__desc'] = data2['Comment']
data3['tag__value'] = data2['value']
data3['tag__unit'] = data2['tag__unit']

### Final DataFrame

In [12]:
data3.head()

,event_timestamp,tag__id,tag__name,tag__desc,tag__value,tag__unit
0,2022-06-05 21:58:33,35831,L_R1SDD_SPDFBK,R1 Screw Down DS Speed Feedback (mps),0,mps
1,2022-06-05 21:58:33,35828,L_R1SDD_SPDREF,R1 Screw Down DS Speed Reference (mps),0,mps
2,2022-06-05 21:58:33,83537,L_R1SDLS_SPDFBK,R1 SCREW DOWN LS SPEED FBK (MPS),0,MPS
3,2022-06-05 21:58:33,83534,L_R1SDLS_SPDREF,R1 SCREW DOWN LOW SPEED REF (MPS),0,MPS
4,2022-06-05 21:58:33,35832,L_R1SDL_SPDFBK,R1 Screw Down LS Speed Feedback (mps),0,mps


### Converting to output csv

In [13]:
data3.to_csv("Excelfiles/Output.csv",index = False)